In [1]:
import requests
import base64

In [2]:
url = "http://35.227.24.107/62ab5b5f90/?post="
cipher = "a!qhpB3ng-tI3UPeNXl8353eAKYet6WLuj!U-Z4qRQj!vmG6TxzUeRxmLVOv5ubMwas1n!J5W73QzcErlIUdPCQF9n25-Uqr-ObGhKOwdIooykbiD5iEyGYUPedRr85xBr7JqLVEZADU0MGBdqdFXt6HEMC9CVggzFqeHuh9sZFmslqOObgsEx3-KEdk2MSl1nZdjGy-P0bV9aiDGJGP5A~~"

In [3]:
b64_ct = cipher.replace('~', '=').replace('!', '/').replace('-', '+')
ct = base64.b64decode(b64_ct)

In [4]:
guess = (ct[:32])
guess = base64.b64encode(guess)
guess = guess.decode('utf-8').replace('=', '~').replace('/', '!').replace('+', '-')
print(guess)

a!qhpB3ng-tI3UPeNXl8353eAKYet6WLuj!U-Z4qRQg~


Add a scenario where you were getting error because the decryption was trying to access data[-1] because we only sent one block of data. And decryption needed 2 blocks.

In [5]:
payload = url + guess
print(payload)
respo = requests.get(payload)
print(respo.text)

http://35.227.24.107/62ab5b5f90/?post=a!qhpB3ng-tI3UPeNXl8353eAKYet6WLuj!U-Z4qRQg~
Traceback (most recent call last):
  File "./main.py", line 69, in index
    post = json.loads(decryptLink(postCt).decode('utf8'))
  File "./common.py", line 49, in decryptLink
    return unpad(cipher.decrypt(data))
  File "./common.py", line 22, in unpad
    raise PaddingException()
PaddingException



## Arranging blocks for padding oracle

In [6]:
cipherBlocks = ([ct[i:i+16] for i in range(0, len(ct), 16)])
print(cipherBlocks)

[b'k\xfa\xa1\xa4\x1d\xe7\x83\xebH\xddC\xde5y|\xdf', b'\x9d\xde\x00\xa6\x1e\xb7\xa5\x8b\xba?\xd4\xf9\x9e*E\x08', b'\xff\xbea\xbaO\x1c\xd4y\x1cf-S\xaf\xe6\xe6\xcc', b'\xc1\xab5\x9f\xf2y[\xbd\xd0\xcd\xc1+\x94\x85\x1d<', b'$\x05\xf6}\xb9\xf9J\xab\xf8\xe6\xc6\x84\xa3\xb0t\x8a', b'(\xcaF\xe2\x0f\x98\x84\xc8f\x14=\xe7Q\xaf\xceq', b'\x06\xbe\xc9\xa8\xb5Dd\x00\xd4\xd0\xc1\x81v\xa7E^', b'\xde\x87\x10\xc0\xbd\tX \xccZ\x9e\x1e\xe8}\xb1\x91', b'f\xb2Z\x8e9\xb8,\x13\x1d\xfe(Gd\xd8\xc4\xa5', b'\xd6v]\x8cl\xbe?F\xd5\xf5\xa8\x83\x18\x91\x8f\xe4']


In [7]:
num_of_blocks = len(cipherBlocks)
index = 15
asItIs_block = cipherBlocks[num_of_blocks-1]
for i in range(0,255):
    mod_iv = b'\x00' * (index) + bytes([i])
    newPost = mod_iv + asItIs_block
    newPost = base64.b64encode(newPost).decode('utf-8').replace('=', '~').replace('/', '!').replace('+', '-')
    payload_url = url + newPost
    payload_respo = requests.get(payload_url)
    if "PaddingException" not in payload_respo.text:
        print(payload_url)
        print("For index: " + str(i) + "\nFor Byte: " + str(bytes([i])))
        break

http://35.227.24.107/62ab5b5f90/?post=AAAAAAAAAAAAAAAAAAAAuyjKRuIPmITIZhQ951GvznE~
For index: 187
For Byte: b'\xbb'


Now we know the padding is correct for the found byte. Time for some cryptography mathematics. Having the prior knowledge of CBC decryption we know that:
```
PlainText[2] = D(K,CipherText[2]) (+) CipherText[1]
```
We start manupilating the last byte of the CipherText[1] here, to retrieve a proper padding. Now if a proper padding is found with one of our guesses, and going by the padding rules in PKCS7, we know that if a single byte's padding was correct, it must've been '0x01'. we know that:
```
D(K,CipherText[2]) (+) modified_CipherText[1] = 0x01
```
Rearrange that and we get:
```
D(K,CipherText[2]) = modified_CipherText[1] (+) 0x01
```
This implies that whatever guess we made (the mod_iv variable), should be XORed with '0x01' in order to obtain the decrypted value. Hence the CipherText[2] was decrypted but failed to convert to a valid utf-8 value.

In [8]:
num_of_blocks = len(cipherBlocks)
index = 15
asItIs_block = cipherBlocks[num_of_blocks-1]
decryptedBlock = bytes()
suffix = bytes()

while index >= 0:
    for i in range(0,255):
        mod_iv = b'\x00' * (index) + bytes([i]) +suffix
        newPost = mod_iv + asItIs_block
        newPost = base64.b64encode(newPost).decode('utf-8').replace('=', '~').replace('/', '!').replace('+', '-')
        payload_url = url + newPost
        payload_respo = requests.get(payload_url)
        if "PaddingException" not in payload_respo.text:
            decryptedBlock = bytes([(16-index) ^ mod_iv[index]]) + decryptedBlock
            index -= 1
            suffix = bytes([b ^ (16-index) for b in decryptedBlock])
            break

Decrypted Block: b'\xaf'
At index: 15
Decrypted Block: b'\xce\xaf'
At index: 14
Decrypted Block: b'\xd2\xce\xaf'
At index: 13
Decrypted Block: b'n\xd2\xce\xaf'
At index: 12
Decrypted Block: b'Mn\xd2\xce\xaf'
At index: 11
Decrypted Block: b'"Mn\xd2\xce\xaf'
At index: 10
Decrypted Block: b'\xf4"Mn\xd2\xce\xaf'
At index: 9
Decrypted Block: b'\x17\xf4"Mn\xd2\xce\xaf'
At index: 8
Decrypted Block: b'\x19\x17\xf4"Mn\xd2\xce\xaf'
At index: 7
Decrypted Block: b'&\x19\x17\xf4"Mn\xd2\xce\xaf'
At index: 6
Decrypted Block: b'\xc5&\x19\x17\xf4"Mn\xd2\xce\xaf'
At index: 5
Decrypted Block: b'\x1b\xc5&\x19\x17\xf4"Mn\xd2\xce\xaf'
At index: 4
Decrypted Block: b'\xf0\x1b\xc5&\x19\x17\xf4"Mn\xd2\xce\xaf'
At index: 3
Decrypted Block: b'$\xf0\x1b\xc5&\x19\x17\xf4"Mn\xd2\xce\xaf'
At index: 2
Decrypted Block: b'\xe3$\xf0\x1b\xc5&\x19\x17\xf4"Mn\xd2\xce\xaf'
At index: 1
Decrypted Block: b'_\xe3$\xf0\x1b\xc5&\x19\x17\xf4"Mn\xd2\xce\xaf'
At index: 0


In [11]:
print(str(decryptedBlock))

b'_\xe3$\xf0\x1b\xc5&\x19\x17\xf4"Mn\xd2\xce\xaf'


In [13]:
def bxor(b1, b2): # use xor for bytes
    result = b""
    for b1, b2 in zip(b1, b2):
        result += bytes([b1 ^ b2])
    return result

result = bxor(cipherBlocks[num_of_blocks-2], decryptedBlock)

In [14]:
print(result)

b'9Q~~"}\n\n\n\n\n\n\n\n\n\n'
